# Criar uma consulta que mostre, nesta ordem: Nome, Estado e Status dos clientes

In [1]:
from src.spark_client import get_spark

spark = get_spark()


In [ ]:
from pyspark.sql.functions import col

df_clients = spark.read.parquet("/data/raw/Clientes.parquet")

df_clients.select(col("Cliente"), col("Estado"), col("Status")).show()

# Criar uma consulta que mostre apenas os clientes do Status "platinum" e "gold"

In [ ]:
df_clients.filter((col("Status") == "Platinum") | (col("Status") == "Gold")).show()